This notebook is meant to clean some issues or formating for some things that have come while creating a Tableau notebook. This notebook is not meant to be a thorough or complete cleaning.

- Delete loans with 'Cancelled' `LoanStatus`, since they weren't issued/originated.
- Assign good and bad loans:
    + Good loans - anything in good standing or up to a month overdue
        - 'Current'
        - 'Completed'
        - 'FinalPaymentInProgress'
        - 'Past Due (1-15 days)'
        - 'Past Due (16-30 days)'
    + Bad loans - defaulted, sold to debt collector, or > 30 days overdue:
        - 'Chargedoff'
        - 'Defaulted',
        - 'Past Due (31-60 days)'
        - 'Past Due (61-90 days)'
        - 'Past Due (91-120 days)'
        - 'Past Due (>120 days)'
- Consolidate 'Not employed' `IncomeRange` to '$0', since it is redundant.
- Create `IncomeRangeOrder` to manually sort the categorical `IncomeRange` in Tableau

In [24]:
import numpy as np
import pandas as pd

In [1]:
!ls

Mentor Chat Review.rtfd
Prosper Loan Data - Variable Definitions.xlsx
Prosper logo.png
cleaning.ipynb
prosper webpage metric.png


In [26]:
prosper_df = pd.read_csv('prosperLoanData.csv')

In [27]:
prosper_df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [28]:
prosper_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
ListingKey                             113937 non-null object
ListingNumber                          113937 non-null int64
ListingCreationDate                    113937 non-null object
CreditGrade                            28953 non-null object
Term                                   113937 non-null int64
LoanStatus                             113937 non-null object
ClosedDate                             55089 non-null object
BorrowerAPR                            113912 non-null float64
BorrowerRate                           113937 non-null float64
LenderYield                            113937 non-null float64
EstimatedEffectiveYield                84853 non-null float64
EstimatedLoss                          84853 non-null float64
EstimatedReturn                        84853 non-null float64
ProsperRating (numeric)                84853 non-null float64
ProsperRating (Alpha) 

* Get rid of loans with `LoanStatus` of 'Cancelled'

In [29]:
prosper_df.LoanStatus.value_counts()

Current                   56576
Completed                 38074
Chargedoff                11992
Defaulted                  5018
Past Due (1-15 days)        806
Past Due (31-60 days)       363
Past Due (61-90 days)       313
Past Due (91-120 days)      304
Past Due (16-30 days)       265
FinalPaymentInProgress      205
Past Due (>120 days)         16
Cancelled                     5
Name: LoanStatus, dtype: int64

In [30]:
# drop cancelled
prosper_df = prosper_df[prosper_df.LoanStatus != 'Cancelled']

In [31]:
prosper_df.LoanStatus.value_counts()

Current                   56576
Completed                 38074
Chargedoff                11992
Defaulted                  5018
Past Due (1-15 days)        806
Past Due (31-60 days)       363
Past Due (61-90 days)       313
Past Due (91-120 days)      304
Past Due (16-30 days)       265
FinalPaymentInProgress      205
Past Due (>120 days)         16
Name: LoanStatus, dtype: int64

In [32]:
prosper_df.LoanStatus.value_counts().index

Index(['Current', 'Completed', 'Chargedoff', 'Defaulted',
       'Past Due (1-15 days)', 'Past Due (31-60 days)',
       'Past Due (61-90 days)', 'Past Due (91-120 days)',
       'Past Due (16-30 days)', 'FinalPaymentInProgress',
       'Past Due (>120 days)'],
      dtype='object')

* Create a column to indicate whether loan is/was 'Good' or 'Bad' from `LoanStatus`

In [33]:
# define good loans as anything on-time, completed, or less than a month overdue
good_loans = ['Current',
              'Completed',
              'FinalPaymentInProgress',
              'Past Due (1-15 days)',
              'Past Due (16-30 days)']
# define bad loans as defaulted, sold to debt collector, or anything more than a month overdue
bad_loans = ['Chargedoff',
             'Defaulted',
             'Past Due (31-60 days)',
             'Past Due (61-90 days)',
             'Past Due (91-120 days)',
             'Past Due (>120 days)']
# define a function to map 'Good' and 'Bad'
def good_or_bad(category):
    if category in good_loans:
        return "Good"
    elif category in bad_loans:
        return "Bad"
    else: return np.nan
# map 'LoanStatus' to 'IsGoodOrBadLoan'
prosper_df['IsGoodOrBadLoan'] = prosper_df.LoanStatus.map(good_or_bad)

In [34]:
prosper_df['IsGoodOrBadLoan'].value_counts()

Good    95926
Bad     18006
Name: IsGoodOrBadLoan, dtype: int64

* Change 'Not employed' to '$0' in `IncomeRange`

In [35]:
prosper_df['IncomeRange'].value_counts()

$25,000-49,999    32191
$50,000-74,999    31050
$100,000+         17337
$75,000-99,999    16916
Not displayed      7737
$1-24,999          7274
Not employed        806
$0                  621
Name: IncomeRange, dtype: int64

In [46]:
# set 'Not employed' to '$0' in `IncomeRange`
prosper_df.loc[prosper_df.IncomeRange == 'Not employed', 'IncomeRange'] = '$0'

In [47]:
prosper_df['IncomeRange'].value_counts()

$25,000-49,999    32191
$50,000-74,999    31050
$100,000+         17337
$75,000-99,999    16916
Not displayed      7737
$1-24,999          7274
$0                 1427
Name: IncomeRange, dtype: int64

In [51]:
sorted(list(prosper_df['IncomeRange'].value_counts().index))

['$0',
 '$1-24,999',
 '$100,000+',
 '$25,000-49,999',
 '$50,000-74,999',
 '$75,000-99,999',
 'Not displayed']

In [52]:
income_order = {'$0': 1,
                '$1-24,999': 2,
                '$25,000-49,999': 3,
                '$50,000-74,999': 4,
                '$75,000-99,999': 5,
                '$100,000+': 6,
                'Not displayed': 0}
prosper_df['IncomeRangeOrder'] = prosper_df.IncomeRange.map(lambda x: income_order[x])

In [53]:
prosper_df.to_csv('prosperLoanData_cleaned.csv')